<a href="https://colab.research.google.com/github/konkall/Fake_News_detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Detection

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')